# Preprocessing

## Loading data / packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import bigfish.detection as detection

RUN_PATH = "/media/floricslimani/SSD4To/SSD_floricslimani/Fish_seq/Davide/2024-08-12 - SeqFISH - HeLa - Puro - R2TP1-2_Run7/"
OUTPUT_PATH = RUN_PATH + "/analysis/density/"
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [2]:
Acquisition = pd.read_feather(RUN_PATH + "/result_tables/Acquisition.feather")
Detection = pd.read_feather(RUN_PATH + "/result_tables/Detection.feather")
Spots = pd.read_feather(RUN_PATH + "/result_tables/Spots.feather")
Gene_map = pd.read_feather(RUN_PATH + "/result_tables/Gene_map.feather")

## Data merge

In [3]:
check_len = len(Detection)
Detection = pd.merge(
    Detection,
    Acquisition.loc[:,['acquisition_id', 'cycle', 'location']],
    on= 'acquisition_id',
    suffixes=('','_acquisition')
)
assert len(Detection) == check_len

Detection = pd.merge(
    Detection,
    Gene_map.loc[:,['cycle','color_id','target']],
    on= ['cycle','color_id']
)
assert len(Detection) == check_len

check_len = len(Spots)
Spots = pd.merge(
    Spots,
    Detection.loc[:,['detection_id', 'acquisition_id', 'target', 'location']],
    on= 'detection_id'
)
assert len(Spots) == check_len


# General clustering

Here we perform DBSCAN algorithm by pulling together spots from all channels so as to identify site of interaction between channels

- can we produce a 3D heatmap showing density of spots per pixel ?
- add new multichannel clusters to viewer
- additional graph analysis

*Note : functional version of this code available in **density.py***

## Post-processing

### Parameters

In [4]:
VOXEL_SIZE = (200,97,97)
MIN_NB_CLUSTER = 4

### Pooling data

In [5]:
spots_coordinates_per_fov = Spots.groupby(['location']).agg({
    'z' : list,
    'y' : list,
    'x' : list,
    })
spots_coordinates_per_fov

,z,y,x
location,,,
Location-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[124, 422, 618, 707, 1477, 1505, 1604, 1840, 1...","[620, 245, 277, 516, 1019, 1037, 1290, 1356, 1..."
Location-02,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 7, 8, 12, 23, 28, 33, 33, 37, 37, 40, 4...","[1005, 1426, 1057, 1015, 1057, 1025, 962, 946,..."
Location-03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 9, 22, 31, 32, 43, 59, 67, 78, 99, 106, 11...","[1276, 1403, 1395, 1344, 1242, 1357, 1077, 135..."
Location-04,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 15, 15, 36, 45, 46, 50, 73, 74, 94, 97, 9...","[901, 166, 921, 175, 183, 174, 76, 187, 473, 8..."
Location-05,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1231, 1247, 1248, 1254, 1257, 1262, 1263, 126...","[1904, 1406, 1374, 1396, 1412, 1367, 1872, 135..."
Location-06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 3, 10, 14, 48, 56, 59, 63, 67, 68, 72, 76,...","[377, 270, 745, 255, 671, 758, 1225, 678, 658,..."
Location-07,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[429, 430, 663, 879, 879, 1046, 1305, 1339, 13...","[887, 920, 937, 1502, 1518, 1553, 1879, 1899, ..."
Location-09,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[11, 14, 44, 59, 61, 81, 86, 89, 97, 98, 106, ...","[1768, 1544, 1708, 1573, 1562, 1610, 1791, 178..."
Location-10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 116, 318, 348, 763, 763, 764, 766, 780, 80...","[1738, 756, 1800, 201, 1510, 1646, 1540, 1661,..."


### DBSCAN

In [6]:
Spots_clustered = pd.DataFrame(columns=['cluster_id','z','y','x', 'location'])
for location in spots_coordinates_per_fov.index : 
    data_selec = spots_coordinates_per_fov.loc[location]

    spots = np.array(
        list(zip(data_selec['z'],data_selec['y'],data_selec['x'],)),
        dtype=int)

    clustered_spot, clusters = detection.detect_clusters(
        spots,
        VOXEL_SIZE,
        max(VOXEL_SIZE),
        MIN_NB_CLUSTER,
    )

    z,y,x, cluster_id = zip(*clustered_spot)
    cluster_z, cluster_y, cluster_x, spot_number, cluster_index = zip(*clusters)

    new_Spots_clustered = pd.DataFrame({
            'cluster_id' : cluster_id,
            'z' : z,
            'y' : y,
            'x' : x,
            'location' : location,
        })
    
    new_Clusters = pd.DataFrame({
        'cluster_id' : cluster_index,
        'cluster_centroid_z' : cluster_z, 
        'cluster_centroid_y' : cluster_y, 
        'cluster_centroid_x' : cluster_x, 
    })

    check_len = len(new_Spots_clustered)
    new_Spots_clustered = pd.merge(
        new_Spots_clustered,
        new_Clusters,
        on='cluster_id',
        how='left',
        validate='m:1'
    )
    assert len(new_Spots_clustered) == check_len , "check for duplication"

    Spots_clustered = pd.concat([
        Spots_clustered,
        new_Spots_clustered,
    ], axis= 0)

Spots_clustered

,cluster_id,z,y,x,location,cluster_centroid_z,cluster_centroid_y,cluster_centroid_x
0,-1,0,124,620,Location-01,NaN,NaN,NaN
1,-1,0,422,245,Location-01,NaN,NaN,NaN
2,-1,0,618,277,Location-01,NaN,NaN,NaN
3,-1,0,707,516,Location-01,NaN,NaN,NaN
4,-1,0,1477,1019,Location-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11444,-1,44,970,1232,Location-10,NaN,NaN,NaN
11445,-1,43,1205,1165,Location-10,NaN,NaN,NaN
11446,-1,43,1204,1166,Location-10,NaN,NaN,NaN
11447,-1,44,1320,1477,Location-10,NaN,NaN,NaN


### Duplicated values for after clustering

This dataframe is expected to contain duplicated values for spots with perfect colocalization. What is important for downstream analysis is that they have the same cluster_id. Next step is to retrieve this cluster id to the main Spots table, to avoid duplication during merge on coordinates, we remove duplicated rows.

In [7]:
Spots_clustered[Spots_clustered.duplicated()]

,cluster_id,z,y,x,location,cluster_centroid_z,cluster_centroid_y,cluster_centroid_x
3667,1,3,133,1673,Location-01,2.0,133.0,1672.0
3674,-1,0,626,272,Location-01,NaN,NaN,NaN
3693,2,5,1028,1021,Location-01,3.0,1026.0,1018.0
3694,2,4,1028,1022,Location-01,3.0,1026.0,1018.0
3695,2,5,1028,1021,Location-01,3.0,1026.0,1018.0
...,...,...,...,...,...,...,...,...
11424,106,43,739,1700,Location-10,43.0,739.0,1700.0
11426,106,43,739,1700,Location-10,43.0,739.0,1700.0
11427,106,44,739,1701,Location-10,43.0,739.0,1700.0
11428,106,43,739,1700,Location-10,43.0,739.0,1700.0


In [8]:
prev_len = len(Spots_clustered)
Spots_clustered = Spots_clustered.drop_duplicates()
print("{0} duplicates dropped.".format(prev_len - len(Spots_clustered)))

12506 duplicates dropped.


### Merging data into main Spots frame

In [9]:
check_len = len(Spots) #No filter/duplication expected.

Spots_clustered = Spots_clustered.rename(columns={'cluster_id' : 'general_cluster_id'})

Spots = pd.merge(
    Spots,
    Spots_clustered,
    on=['location','z','y','x'],
    validate= 'm:1'
)

assert len(Spots) == check_len
Spots

,spot_id,cluster_id,drifted_z,drifted_y,drifted_x,intensity,population,detection_id,acquisition_id_x,drift_z,...,z,y,x,acquisition_id_y,target,location,general_cluster_id,cluster_centroid_z,cluster_centroid_y,cluster_centroid_x
0,0,NaN,0,124,620,8747,free,1,0,0,...,0,124,620,0,POLR2A,Location-01,-1,NaN,NaN,NaN
1,1,NaN,0,422,245,8427,free,1,0,0,...,0,422,245,0,POLR2A,Location-01,-1,NaN,NaN,NaN
2,2,NaN,0,618,277,11475,free,1,0,0,...,0,618,277,0,POLR2A,Location-01,-1,NaN,NaN,NaN
3,3,NaN,0,707,516,8180,free,1,0,0,...,0,707,516,0,POLR2A,Location-01,-1,NaN,NaN,NaN
4,4,NaN,0,1477,1019,9797,free,1,0,0,...,0,1477,1019,0,POLR2A,Location-01,-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299188,299188,NaN,44,969,1233,5129,free,251,124,0,...,44,970,1232,124,AGO1,Location-10,-1,NaN,NaN,NaN
299189,299189,NaN,43,1204,1166,4354,free,251,124,0,...,43,1205,1165,124,AGO1,Location-10,-1,NaN,NaN,NaN
299190,299190,NaN,43,1203,1167,3982,free,251,124,0,...,43,1204,1166,124,AGO1,Location-10,-1,NaN,NaN,NaN
299191,299191,NaN,44,1319,1478,4350,free,251,124,0,...,44,1320,1477,124,AGO1,Location-10,-1,NaN,NaN,NaN


## Analysis

### Data grouping

In [10]:
Clustered_spots = Spots.loc[Spots['general_cluster_id'] != -1]

multichannel_clusters = Clustered_spots.groupby(['location', 'general_cluster_id']).aggregate({
    'spot_id' : 'count',
    'target' : ['nunique','unique'],
    'cluster_centroid_z' : 'first',
    'cluster_centroid_y' : 'first',
    'cluster_centroid_x' : 'first',
})

multichannel_clusters.to_excel(OUTPUT_PATH + "Multichannel_cluster.xlsx")